# Week 7 status

План:
1. Список влитых PR
2. Результат работы скомпилированного кода
3. Список фичей на следующую неделю

## Влитые PR в master.
1. [path_context_reader -> master](https://github.com/bmstu-iu8-g4-2020-project/code2var/pull/16)

Написан файл, отвечающий за подготовку датасета для нейросети code2vec. Потенциально легко адаптируем для code2var.

## Демонстрация работы программы

### Построение датасета

In [1]:
%cd ../

/home/rkulagin/university/IU8-8-3/TiMP-course-project/code/code2vec


In [2]:
import tensorflow as tf
import tensorboard
import os
import datetime
import config
from vocabulary import Code2VecVocabs
from path_context_reader import PathContextReader

%load_ext tensorboard


In [3]:
config.config.CREATE_VOCAB = True
config.config.TRAINING_FREQ_DICTS_PATH = "dataset/java-small/java-small.c2v.dict"
c2v_vocabs = Code2VecVocabs()
pcr = PathContextReader(is_train=True, vocabs=c2v_vocabs,
                        csv_path="dataset/java-small/java-small.train_vec.csv")
dataset = pcr.get_dataset()


Creating vocab from dataset/java-small/java-small.c2v.dict
Loading frequency dicts from dataset/java-small/java-small.c2v.dict
Loading token freq dict
Loading path freq dict
Loading target freq dict
Creating token vocab
Creating vocab from frequency dictionary of 130 elements
Created token vocab
Creating path vocab
Creating vocab from frequency dictionary of 911 elements
Created path vocab
Creating target vocab
Creating vocab from frequency dictionary of 18 elements
Created target vocab
Created all vocabs


### Построение графа для tf.function _generate_input_tensors


In [4]:
dataset = tf.data.experimental.CsvDataset("dataset/java-small/java-small.train_vec.csv", [""] * (
        config.config.MAX_CONTEXTS + 1), field_delim=" ",
                                          use_quote_delim=False)

# Code below have been taken from https://www.tensorflow.org/tensorboard/graphs#graphs_of_tffunctions
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'weekly_reports/logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.

for line in dataset:
    pcr._generate_input_tensors(*line)
    break

with writer.as_default():
  tf.summary.trace_export(
      name="generate_input_tensors_trace",
      step=0,
      profiler_outdir=logdir)


Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


In [5]:
%tensorboard --logdir weekly_reports/logs/func/